# Python Note 100 - File

```
:date: 2017-02-13
:modified: 2024-03-04
:slug: python-note-100-file
:tags: python, note, file
:category: Development
:author: Dormouse Young
:summary: Python note series 100 - file
```

In [1]:
import os
import stat
from collections import Counter
from datetime import datetime
from pathlib import Path

## 创建文件

In [2]:
my_file = Path('/tmp/first/firstone/tmp.txt')

# my_file.touch()
# touch 方法用于创建空文件，目录必须存在，否则无法创建
# ---------------------------------------------------------------------------
# FileNotFoundError                         Traceback (most recent call last)
# Cell In[9], line 2
#       1 my_file = Path('/tmp/first/firstone/tmp.txt')
# ----> 2 my_file.touch()
# ....

In [3]:
my_path = Path('/tmp/first/firstone/')
my_path.mkdir(exist_ok=True,parents=True)
my_file.touch()
my_file.exists()

True

## 文件名的拆解

In [4]:
my_file.name                          # 获取文件名

'tmp.txt'

In [5]:
my_file.stem                          # 获取文件名除后缀的部分

'tmp'

In [6]:
Path('tmp_file.tar.gz').stem  # 获取文件名除后缀的部分

'tmp_file.tar'

In [7]:
my_file.suffix                        # 文件后缀

'.txt'

In [8]:
my_file.suffixes                 # 文件的后缀们...

['.txt']

In [9]:
Path('tmp_file.tar.gz').suffix  # 文件后缀

'.gz'

In [10]:
Path('tmp_file.tar.gz').suffixes # 文件的后缀们...

['.tar', '.gz']

In [11]:
my_file.parent                        # 相当于dirnanme

PosixPath('/tmp/first/firstone')

In [12]:
# p.parents                       # 返回一个iter, 包含所有父目录
list(my_file.parents)

[PosixPath('/tmp/first/firstone'),
 PosixPath('/tmp/first'),
 PosixPath('/tmp'),
 PosixPath('/')]

In [13]:
my_file.parts                     # 将路径通过分隔符分割成一个元组

('/', 'tmp', 'first', 'firstone', 'tmp.txt')

```python
>>> desk = Path('C:/Users/Administrator/Desktop/')
>>> desk.parent
WindowsPath('C:/Users/Administrator')

>>> desk.parent.parent
WindowsPath('C:/Users')

>>> list(desk.parents)
[WindowsPath('C:/Users/Administrator'),
 WindowsPath('C:/Users'),
 WindowsPath('C:/')]
```

## 文件名替换

In [14]:
# with_name(name)替换路径最后一部分并返回一个新路径
my_file.with_name('python.txt')

PosixPath('/tmp/first/firstone/python.txt')

In [15]:
# with_suffix(suffix)替换扩展名，返回新的路径，扩展名存在则不变
my_file.with_suffix('.txt')

PosixPath('/tmp/first/firstone/tmp.txt')

## 文件信息

In [16]:
my_file.stat()                        # 获取详细信息

os.stat_result(st_mode=33204, st_ino=30540110, st_dev=2050, st_nlink=1, st_uid=1000, st_gid=1000, st_size=0, st_atime=1721026452, st_mtime=1721026452, st_ctime=1721026452)

In [17]:
my_file.stat().st_size                # 文件大小

0

In [18]:
my_file.stat().st_ctime               # 创建时间

1721026452.1530297

In [19]:
my_file.stat().st_mtime               # 修改时间

1721026452.1530297

In [20]:
# 以下为老方法

my_file_str = str(my_file)
oct(stat.S_IMODE(os.lstat(my_file_str).st_mode))

'0o664'

In [21]:
oct(os.stat(my_file_str)[stat.ST_MODE])

'0o100664'

In [22]:
oct(os.stat(my_file_str).st_mode & 0o777)

'0o664'

## 读写文件

### 写入文件

In [23]:
todo_string="""# TODO LIST
## Today

* Read book
* Buy milk

## Tomorrow

* Hike out
"""
todo_file_path = Path('/tmp/todo.md')
todo_file_path.write_text(todo_string)

70

### 读取文件

In [24]:
content = todo_file_path.read_text(encoding="utf-8")
[line for line in content.splitlines() if line.startswith("*")]

['* Read book', '* Buy milk', '* Hike out']

## 复制文件

In [25]:
# Pathlib 没有现成的复制，只有用读取和写入替代

# 可以考虑使用老的 shutil ，下文有示例

my_file = Path('/tmp/first/firstone/tmp.txt')
des_file = my_file.with_name('python.txt')
des_file.write_bytes(my_file.read_bytes())
[line for line in des_file.read_bytes().splitlines()]

[]

## 移动文件（包含重命名文件）

In [26]:
source = Path('/tmp/first/firstone/tmp.txt')
destination = Path('/tmp/first/first_tmp.txt')

if not destination.exists():
    source.replace(destination)

# 为了避免 race condition ,可以采用以下方式
try:
    with destination.open(mode="xb") as file:
        file.write(source.read_bytes())
except FileExistsError:
    print(f"File {destination} exists already.")
else:
    source.unlink()

File /tmp/first/first_tmp.txt exists already.


## 文件操作专题

### 遍历文件

In [27]:
paths = [
    '/tmp/iterfile/oneone.txt',
    '/tmp/iterfile/onetwo.txt',
    '/tmp/iterfile/twoone.py',
    '/tmp/iterfile/sub/subone.py',
]
for item in paths:
    my_file = Path(item)
    my_file.parent.mkdir(exist_ok=True,parents=True)
    if not my_file.exists():
        my_file.touch()
Counter(path.suffix for path in Path('/tmp/iterfile/').iterdir())
# 这里注意子文件夹里的文件是不涉及的。子文件夹没有扩张名，也会计数。

Counter({'.txt': 2, '': 1, '.py': 1})

In [28]:
Counter( Path('/tmp/iterfile/').iterdir())

Counter({PosixPath('/tmp/iterfile/oneone.txt'): 1,
         PosixPath('/tmp/iterfile/onetwo.txt'): 1,
         PosixPath('/tmp/iterfile/sub'): 1,
         PosixPath('/tmp/iterfile/twoone.py'): 1})

In [29]:
# 用 glob 可以排除子目录
Counter(path.suffix for path in Path('/tmp/iterfile/').glob('*.*'))

Counter({'.txt': 2, '.py': 1})

In [30]:
# 用 rglob 可以递归子目录
Counter(path.suffix for path in Path('/tmp/iterfile/').rglob('*.*'))

Counter({'.txt': 2, '.py': 2})

### 显示树形目录结构

In [31]:
def tree(directory):
    print(f"+ {directory}")
    for path in sorted(directory.rglob("*")):
        depth = len(path.relative_to(directory).parts)
        spacer = "    " * depth
        print(f"{spacer}+ {path.name}")
tree( Path('/tmp/iterfile/'))

+ /tmp/iterfile
    + oneone.txt
    + onetwo.txt
    + sub
        + subone.py
    + twoone.py


### 查找最新修改的文件

In [32]:
my_dir = Path('/tmp/iterfile/')
time, file_path = max((f.stat().st_mtime, f) for f in my_dir.iterdir())
datetime.fromtimestamp(time), file_path

(datetime.datetime(2024, 7, 15, 14, 54, 12, 373026),
 PosixPath('/tmp/iterfile/twoone.py'))

## 以前的老方法

### 打开文件

    with open("/tmp/foo.txt") as file:
        data = file.read()

    with open('examples/favorite-people.txt', encoding='utf-8') as a_file:
        for a_line in a_file:
            line_number += 1
            print('{:>4} {}'.format(line_number, a_line.rstrip()))

使用字符串的 format() 方法可以打印出行号和行自身。格式说明符 {:>4} 的意思是
“使用最多四个空格使之右对齐，然后打印此参数。”变量 a_line 是包括回车符等在
内的完整的一行。字符串方法rstrip()可以去掉尾随的空白符，包括回车符。

### 写入文件


    with open(csvfile, 'w') as f:
        f.writelines(linelist)
    f.close()

### 关于 open 模式

open 的模式如下表：

|命令| 说明 |
|----|------|
|r   |以读方式打开|
|w   |以写方式打开|
|a   |以追加模式打开 (从 EOF 开始, 必要时创建新文件)|
|r+  |以读写模式打开|
|w+  |以读写模式打开 (参见 w )|
|a+  |以读写模式打开 (参见 a )|
|rb  |以二进制读模式打开|
|wb  |以二进制写模式打开 (参见 w )|
|ab  |以二进制追加模式打开 (参见 a )|
|rb+ |以二进制读写模式打开 (参见 r+ )|
|wb+ |以二进制读写模式打开 (参见 w+ )|
|ab+ |以二进制读写模式打开 (参见 a+ )|


### shutil 操作

复制文件：

* shutil.copyfile("oldfile","newfile") oldfile 和 newfile 都只能是文件。
* shutil.copy("oldfile","newfile") oldfile 只能是文件夹， newfile 可以是文件，
  也可以是目标目录

复制文件夹：

* shutil.copytree("olddir","newdir") olddir和newdir都只能是目录，且newdir必须不存在

移动文件（目录）：

* shutil.move("oldpos","newpos")

删除目录：

* shutil.rmtree("dir")    空目录、有内容的目录都可以删

### 相关函数


* fp.read([size]) #size为读取的长度，以byte为单位
* fp.readline([size]) #读一行，如果定义了size，有可能返回的只是一行的一部分
* fp.readlines([size]) #把文件每一行作为一个list的一个成员，并返回这个list。其实它的内部是通过循环调用readline()来实现的。如果提供size参数，size是表示读取内容的总长，也就是说可能只读到文件的一部分。
* fp.write(str) #把str写到文件中，write()并不会在str后加上一个换行符
* fp.writelines(seq) #把seq的内容全部写到文件中(多行一次性写入)。这个函数也只是忠实地写入，不会在每行后面加上任何东西。
* fp.close() #关闭文件。python会在一个文件不用后自动关闭文件，不过这一功能没有保证，最好还是养成自己关闭的习惯。  如果一个文件在关闭后还对其进行操作会产生ValueError
* fp.flush() #把缓冲区的内容写入硬盘
* fp.fileno() #返回一个长整型的”文件标签“
* fp.isatty() #文件是否是一个终端设备文件（unix系统中的）
* fp.tell() #返回文件操作标记的当前位置，以文件的开头为原点
* fp.next() #返回下一行，并将文件操作标记位移到下一行。把一个file用于for … in file这样的语句时，就是调用next()函数来实现遍历的。
* fp.seek(offset[,whence]) #将文件打操作标记移到offset的位置。这个offset一般是相对于文件的开头来计算的，一般为正数。但如果提供了whence参数就不一定了，whence可以为0表示从头开始计算，1表示以当前位置为原点计算。2表示以文件末尾为原点进行计算。需要注意，如果文件以a或a+的模式打开，每次进行写操作时，文件操作标记会自动返回到文件末尾。
* fp.truncate([size]) #把文件裁成规定的大小，默认的是裁到当前文件操作标记的位置。如果size比文件的大小还要大，依据系统的不同可能是不改变文件，也可能是用0把文件补到相应的大小，也可能是以一些随机的内容加上去。

### os 和 os.path 模块

* os.mkdir("file")：创建目录
* os.rmdir("dir")：只能删除空目录
* os.listdir(dirname)：列出dirname下的目录和文件
* os.getcwd()：获得当前工作目录
* os.curdir：返回当前目录（'.')
* os.chdir(dirname)：改变工作目录到dirname
* os.remove("file")：删除文件
* os.rename("oldname","newname")：重命名文件（目录），文件或目录都是使用这条命令
* os.path.isdir(name)：判断name是不是一个目录，name不是目录就返回false
* os.path.isfile(name)：判断name是不是一个文件，不存在name也返回false
* os.path.exists(name)：判断是否存在文件或目录name
* os.path.getsize(name)：获得文件大小，如果name是目录返回0L
* os.path.abspath(name)：获得绝对路径
* os.path.normpath(path)：规范path字符串形式
* os.path.split(name)：分割文件名与目录（事实上，如果你完全使用目录，它也
  会将最后一个目录作为文件名而分离，同时它不会判断文件或目录是否存在）
* os.path.splitext()：分离文件名与扩展名，返回一个tuple：("aaa",".txt")
* os.path.join(path,name)：连接目录与文件名或目录
* os.path.basename(path)：返回文件名
* os.path.dirname(path)：返回文件路径

### 获得同一后缀名的文件


    import glob
    for filename in glob.glob("*.xls"):
         print filename